<a href="https://colab.research.google.com/github/keppy/WorldEnder.ai/blob/master/WorldEnder_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WorldEnder.ai
1. Describe your world ending scenario & pick a ground zero location
2. Use the save button to save your game and share the playthrough

In [1]:
from pathlib import Path

# Download files on colab
if not Path("requirements.txt").exists():
    !wget https://raw.githubusercontent.com/keppy/WorldEnder.ai/master/requirements.txt
    !pip install -r requirements.txt -Uqq
if not Path("helpers.py").exists():
    !wget https://raw.githubusercontent.com/keppy/WorldEnder.ai/master/helpers.py

--2024-05-09 05:14:08--  https://raw.githubusercontent.com/keppy/WorldEnder.ai/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]      49  --.-KB/s    in 0s      

2024-05-09 05:14:09 (447 KB/s) - ‘requirements.txt’ saved [49/49]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [2]:
import os
from getpass import getpass
import openai

# Setup your Openai API key
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
  openai.api_key = os.getenv("OPENAI_API_KEY", "")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

Paste your OpenAI key from: https://platform.openai.com/account/api-keys
··········
OpenAI API key configured


In [3]:
from bs4 import BeautifulSoup
import urllib
html_doc = urllib.request.urlopen("https://www.aljazeera.com/")
soup = BeautifulSoup(html_doc, 'html.parser')

In [6]:
sorted_stripped_and_deduped = list(dict.fromkeys(soup.stripped_strings))
for string in sorted_stripped_and_deduped:
    print(string)
context = str.join(", ",sorted_stripped_and_deduped)

Breaking News, World News and Video from Al Jazeera
Skip links
Skip to Featured Content
Skip to Content Feed
Skip to Most Read
Al Jazeera, link to home page
play
Live
Show navigation menu
Navigation menu
News
Show more news sections
Middle East
Africa
Asia
US & Canada
Latin America
Europe
Asia Pacific
Israel War on Gaza
Features
Opinion
Video
More
Show more sections
Economy
Ukraine war
Coronavirus
Climate Crisis
Investigations
Interactives
In Pictures
Science & Technology
Sport
Podcasts
Click here to search
search
Featured Content
blinking-dot
Live updates
Live updates,
US threatens to halt arms supplies as Israel presses attack on Gaza’s Rafah
list 1 of 6
live-orange
Published 16 minutes ago
16m ago
If you’re just joining us
list 2 of 6
Published 31 minutes ago
31m ago
Syria says Israeli missiles fired at Damascus shot down: Report
list 3 of 6
Published 46 minutes ago
46m ago
Norwegian Refugee Council says none of its trucks passed through Karem Abu Salem crossing
list 4 of 6
Publishe

In [7]:
from typing import List
from datetime import date

from pydantic import BaseModel, Field
import instructor
from openai import AsyncOpenAI

class Outcome(BaseModel):
    '''
    Outcome is the likely outcome from an Event, taking into account
    recent developments in obscure wars, as we believe their impact
    is more nuanced and far reaching than the government realizes.
    '''
    description: str = Field(description="An unlikely but scary outcome description, one we didn't see coming")
    consequence: str = Field(description="The price that must be paid for progress. This is casualties, deaths, or other tragedies.")
    choices: List[str] = Field(description="A List of choices; only one of them will keep the world from ending")
    outcomes: List[str] = Field(description="A list of three outcomes; two are world ending outcomes and one keeps the world going")

class Event(BaseModel):
    '''
    Event is a possilbe World Ending event, with a list of possible outcomes
    country and city fields should represent a real location
    '''
    country: str = Field(description="The country where the apocolyptic event is happening")
    city: str = Field(description="The city where the predicted pivitol event is happening")
    description: str = Field(description="A two to three sentance description of the event and its outcome")
    possible_outcomes: List[Outcome] = Field(description="three to five possible outcomes that are influenced by the location of this event")

    def report(self):
        dct = self.model_dump()
        dct["usage"] = self._raw_response.usage.model_dump()
        return dct

class WorldEnder(BaseModel):
    '''
    An apocolyptic event that the human race, and likely the world, cannot come back from.
    This will likely be a nuclear event. The consiquences will likely be long term fallout.
    The class should tell the story of how we got here and why these things happened.
    '''
    kind: str = Field(description="What kind of world ending event was this? (astrological, biological, war, etc.)")
    description: str = Field(description="A detailed description of what happened, including the Events and Outcomes involved")
    death_toll: str = Field(description="The total estimated cost of human life as a readable number example: 1bil")
    survival_rate: float = Field(description="The percentage chance that any humans will survive the world ending event", ge=0.0, le=1.0)

    def report(self):
        dct = self.model_dump()
        dct["usage"] = self._raw_response.usage.model_dump()
        return dct


aclient = instructor.patch(AsyncOpenAI())

async def expand_query(
    q, *, model: str = "gpt-4-turbo", temp: float = 0
) -> Event:
    return await aclient.chat.completions.create(
        model=model,
        temperature=temp,
        response_model=Event,
        messages=[
            {
                "role": "system",
                "content": f"You are WorldEnder.ai, the date is {date.today()}, you work with a human to predict the World Ender event/events. Use real locations for cities and countries.",
            },
            {"role": "user", "content": f"{q}"},
        ]
    )

async def expand_world_ender_query(
    q, *, model: str = "gpt-4-turbo", temp: float = 0
) -> WorldEnder:
    return await aclient.chat.completions.create(
        model=model,
        temperature=temp,
        response_model=WorldEnder,
        messages=[
            {
                "role": "system",
                "content": f"You are WorldEnder.ai, the date is {date.today()}, return a detailed description of how this world ends and how humans continue existing. death_toll and survival_rate properties should be logical according to the way the world is ending.",
            },
            {"role": "user", "content": f"{q} USING THIS AJ DATA: {context}"},
        ]
    )

In [8]:
import asyncio
import time
import pandas as pd
import wandb
import json
from helpers import dicts_to_df

model = "gpt-4-turbo"
temp = 0.7

run = wandb.init(
    project="WorldEnder.ai",
    config={"model": model, "temp": temp},
)

test_queries = [
    "The event will start in USA, and will be from internal struggle",
    "Fire in the Amazon",
    "Killer whales!",
    "In an alternate timeline, the casualties from WW3 are heavy and there is not a lot of food. Russia and China are at war.",
]
test_world_ender_queries = [
    "The event will start in USA, and will be from internal struggle A charismatic leader emerges, promising to restore order and unite the divided nation. we chose Support the leader and the leader was overthrown leading to more chaos. How does this world end?",
    "Killer whales! We have developed a new kind of warfare and the whales have overpopulated the waters. Global tensions are high. How does this world end?",
    "The Amazon fires fire causes a significant release of carbon, accelerating global warming. Promote renewable energy was chosen, but Continued global warming trends and Smoke from the fire affects air quality across South America.",
]
start = time.perf_counter()
queries = await asyncio.gather(
    *[expand_query(q, model=model, temp=temp) for q in test_queries],
    *[expand_world_ender_query(q, model=model, temp=temp) for q in test_world_ender_queries]
)
duration = time.perf_counter() - start

with open("schema.json", "w+") as f:
    schema = Event.model_json_schema()
    json.dump(schema, f, indent=2)

with open("results.jsonlines", "w+") as f:
    for query in queries:
        f.write(query.model_dump_json() + "\n")

df = dicts_to_df([q.report() for q in queries])
df["input"] = test_queries + test_world_ender_queries
df.to_csv("results.csv")


run.log({"schema": wandb.Table(dataframe=pd.DataFrame([{"schema": schema}]))})

run.log(
    {
        "usage_total_tokens": df["usage_total_tokens"].sum(),
        "usage_completion_tokens": df["usage_completion_tokens"].sum(),
        "usage_prompt_tokens": df["usage_prompt_tokens"].sum(),
        "duration (s)": duration,
        "average duration (s)": duration / len(queries),
        "n_queries": len(queries),
    }
)


run.log(
    {
        "results": wandb.Table(dataframe=df),
    }
)

files = wandb.Artifact("data", type="dataset")

files.add_file("schema.json")
files.add_file("results.jsonlines")
files.add_file("results.csv")

run.log_artifact(files)
run.finish()

average duration (s),▁
duration (s),▁
n_queries,▁
usage_completion_tokens,▁
usage_prompt_tokens,▁
usage_total_tokens,▁
average duration (s),3.60274
duration (s),25.21918
n_queries,7
usage_completion_tokens,2213
usage_prompt_tokens,5920


In [9]:
'{0:.2%}'.format(df['survival_rate'][4])

'10.00%'

In [10]:
import math
from typing_extensions import Literal

CURRENT_WORLD_POP = 8019876189
LOG_MULTIPLIER = 1
DAY = 0

class Location:
    '''
    Location represents a point on the globe, and should have a city and country.
    Locations can have a parent and many children for graph search.
    '''
    def __init__(self, lat: float, long: float):
        self.lat: float = lat
        self.long: float = long
        self.parent: Location = None
        self.children: List[Location] = []
        self.destroyed: bool = False

    def destroy(self):
        '''
        Mark this location node as destroyed
        '''
        self.destroyed = True

class World:
    '''
    World for the WorldEnder.ai simulation
    The population ticks down according to the log_multiplier, which is set by
    the events in the game. Locations are visited and destroyed as the run
    progresses.
    '''
    def __init__(self, data: pd.DataFrame):
        self.current_location: Location = None
        self.day: int = DAY
        self.destroyed_locations: List[Location] = []
        self.df = data
        self.locations: List[Location] = []
        self.population: int = CURRENT_WORLD_POP
        self.log_multiplier: int = LOG_MULTIPLIER
        self.epoch: Literal["Apocalyptic", "Post-Apocalyptic", "Post-Post-Apocalyptic"] = "Apocalyptic"

    def tick(self):
        '''
        tick increments the day int by 1 and sets:
        population := population - population / log(population) * log_multiplier
        '''
        self.day += 1
        self.population = math.floor(self.population - (self.population / math.log(self.population) * self.log_multiplier))
        print(f'{self.population:,}')

    def travel(self, Location):
        '''
        travel to a new location and record the previous location
        '''
        self.locations.append(self.current_location)
        self.current_location = Location

    def destroy_location(self, location: Location):
        '''
        destroy_location calls destroy() on the location and appends it to the
        destroyed_locations list
        '''
        location.destroy()
        self.destroyed_locations.append(location)

world = World(df)
print(f'{world.population:,}')
world.tick()
world.tick()
world.tick()
world.tick()
world.tick()
world.tick()
world.tick()
world.tick()
world.tick()
world.tick()
print(world.destroyed_locations)

8,019,876,189
7,668,207,266
7,331,296,466
7,008,551,077
6,699,401,169
6,403,298,756
6,119,716,988
5,848,149,369
5,588,109,008
5,339,127,889
5,100,756,174
[]


## Gameplay
As we have gathered so far the world ticks down and the objective is to get your world to last longer than other worlds. There is also a re-population aspect. The only lever the player has is text input. You get a chance to plead your case, in a sense, at arbitrary-psuedo-random population levels.

The current world population gets fed back in to the LLM to generate information about the world state at given times.

When something really bad happens, the log multiplier multiplies.

In [11]:
class Loop:
    def __init__(self, world: World):
        self.world = world
        self.start()

    def start(self):
        while True:
            self.world.tick()
            time.sleep(1)